In [ ]:
#!pip install google_play_scraper
#!pip install schedule
#!pip install requests_cache
#!pip install retry_requests
#!pip install hopsworks==4.1.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.1/640.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: hopsworks
    Found existing installation: hopsworks 4.1.0
    Uninstalling hopsworks-4.1.0:
      Successfully uninstalled hopsworks-4.1.0


In [18]:
import sys
sys.path.append('../')
from google_play_scraper import reviews
import schedule
import time
import sys
import csv

In [19]:
def save_filtered_reviews_to_csv(app_id, lang, country, count, output_file):
    # Fetch reviews
    result, _ = reviews(
        app_id,
        lang=lang,
        country=country,
        count=count
    )

    # Specify relevant keys for the CSV
    relevant_keys = ['reviewId', 'content', 'score', 'at', 'thumbsUpCount']

    # Filter the review data to include only relevant keys
    filtered_result = [
        {key: review[key] for key in relevant_keys if key in review}
        for review in result
    ]

    # Write to CSV
    with open(output_file, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=relevant_keys)
        writer.writeheader()  # Write the header row
        writer.writerows(filtered_result)  # Write the filtered review data

    print(f"Saved {len(filtered_result)} filtered reviews to {output_file}")


try :
    save_filtered_reviews_to_csv(
    app_id='com.tinder',  # Replace with the app's package name
    lang='en',                  # Language
    country='us',               # Country
    count=30000,                 # Number of reviews to fetch
    output_file='filtered_reviews.csv'  # Output CSV file
)

except Exception as e:
    print(f"Error during scraping: {e}")

Saved 30000 filtered reviews to filtered_reviews.csv


In [20]:
import pandas as pd

df = pd.read_csv('filtered_reviews.csv')
df.head()

,reviewId,content,score,at,thumbsUpCount
0,55beec0f-d4a8-408f-97d0-7801929933d6,How many times does a girl need to do this fac...,2,2025-01-09 11:37:12,0
1,43e5f03c-c1c7-41d5-94d9-7aec820cac08,This is a very bad app because my girlfriend d...,1,2025-01-09 11:18:00,0
2,34b615f8-082f-4b01-a9fe-f5b389835cfb,good,5,2025-01-09 11:17:16,0
3,3e8c6b60-f72e-410c-a66d-1af42e6672a4,Female confidence machine...,2,2025-01-09 11:11:26,0
4,93cd2627-2c9b-4094-b329-548a04a13432,I recently purchased a package but to my surpr...,2,2025-01-09 11:08:56,2


In [21]:
import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
from functions import util
import os
import warnings
warnings.filterwarnings("ignore")

In [22]:
#monthly_review_df = util.get_periodic_reviews('com.tinder', 'en', 'us', '2024-12-01', '2024-12-31')
#monthly_review_df.head()

In [23]:
df.dropna(inplace=True)
df

,reviewId,content,score,at,thumbsUpCount
0,55beec0f-d4a8-408f-97d0-7801929933d6,How many times does a girl need to do this fac...,2,2025-01-09 11:37:12,0
1,43e5f03c-c1c7-41d5-94d9-7aec820cac08,This is a very bad app because my girlfriend d...,1,2025-01-09 11:18:00,0
2,34b615f8-082f-4b01-a9fe-f5b389835cfb,good,5,2025-01-09 11:17:16,0
3,3e8c6b60-f72e-410c-a66d-1af42e6672a4,Female confidence machine...,2,2025-01-09 11:11:26,0
4,93cd2627-2c9b-4094-b329-548a04a13432,I recently purchased a package but to my surpr...,2,2025-01-09 11:08:56,2
...,...,...,...,...,...
29995,86de20dd-0864-49d0-a760-a95225a1f601,Everything is paid don't waste your data 👎🏻,1,2024-04-26 18:47:17,0
29996,8ee2709d-d8d5-4d5a-bb01-39b2a6758491,The worst dating app by far paid for platinum ...,1,2024-04-26 18:46:45,0
29997,ff52cfc4-5862-45ab-b48f-746750a3713b,Very Friendly and Beautiful woman,5,2024-04-26 18:16:38,0
29998,585d76e0-ab0a-4439-8d6f-309c30fbd392,It's a Spam,1,2024-04-26 18:16:37,0


In [24]:
# Add the 'augmented_comment' column
df['augmented_comment'] = df['content'] + '. Score given on a scale of 1 to 5 is: ' + df['score'].astype(str)

In [25]:
from transformers import pipeline

# Initialize the sentiment classification pipeline
sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student",
    return_all_scores=True
)

# Define a function to classify sentiment and extract the top label
def classify_sentiment(text):
    # Get sentiment scores
    results = sentiment_classifier(text)[0]

    # Extract the label with the highest score
    top_label = max(results, key=lambda x: x['score'])['label']
    return top_label

# Apply sentiment classification to each row in the 'augmented_comment' column
df['sentiment'] = df['augmented_comment'].apply(classify_sentiment)


Device set to use cuda:0


In [26]:
df

,reviewId,content,score,at,thumbsUpCount,augmented_comment,sentiment
0,55beec0f-d4a8-408f-97d0-7801929933d6,How many times does a girl need to do this fac...,2,2025-01-09 11:37:12,0,How many times does a girl need to do this fac...,negative
1,43e5f03c-c1c7-41d5-94d9-7aec820cac08,This is a very bad app because my girlfriend d...,1,2025-01-09 11:18:00,0,This is a very bad app because my girlfriend d...,negative
2,34b615f8-082f-4b01-a9fe-f5b389835cfb,good,5,2025-01-09 11:17:16,0,good. Score given on a scale of 1 to 5 is: 5,positive
3,3e8c6b60-f72e-410c-a66d-1af42e6672a4,Female confidence machine...,2,2025-01-09 11:11:26,0,Female confidence machine.... Score given on a...,positive
4,93cd2627-2c9b-4094-b329-548a04a13432,I recently purchased a package but to my surpr...,2,2025-01-09 11:08:56,2,I recently purchased a package but to my surpr...,negative
...,...,...,...,...,...,...,...
29995,86de20dd-0864-49d0-a760-a95225a1f601,Everything is paid don't waste your data 👎🏻,1,2024-04-26 18:47:17,0,Everything is paid don't waste your data 👎🏻. S...,neutral
29996,8ee2709d-d8d5-4d5a-bb01-39b2a6758491,The worst dating app by far paid for platinum ...,1,2024-04-26 18:46:45,0,The worst dating app by far paid for platinum ...,negative
29997,ff52cfc4-5862-45ab-b48f-746750a3713b,Very Friendly and Beautiful woman,5,2024-04-26 18:16:38,0,Very Friendly and Beautiful woman. Score given...,positive
29998,585d76e0-ab0a-4439-8d6f-309c30fbd392,It's a Spam,1,2024-04-26 18:16:37,0,It's a Spam. Score given on a scale of 1 to 5 ...,negative


In [27]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "6xLHrUPek6nw9STy.ueYbHCnUwYhFQ5xzscRiONFQ4ZmgUxMFUt8bnCu3UX69ejYcaIxeUHynvrMhcrt5"

project = hopsworks.login()
fs = project.get_feature_store()

2025-01-10 11:55:09,023 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-10 11:55:09,039 INFO: Initializing external client
2025-01-10 11:55:09,041 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-10 11:55:10,426 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1185941


In [28]:
reviews_fg = fs.get_or_create_feature_group(
    name='reviews',
    description='Reviews of tinder historically',
    version=1,
    primary_key=['reviewId'],
    event_time="at",
    online_enabled = True,
    statistics_config = {'histograms':True, 'correlations':True},
)

In [29]:
# Reformatting columns
df['at'] = pd.to_datetime(df['at'])  # Convert 'at' to datetime format
df['augmented_comment'] = df['augmented_comment'].astype(str)  # Convert 'augmented_comment' to string
df['sentiment'] = df['sentiment'].astype(str)  # Convert 'sentiment' to string
df['content'] = df['content'].astype(str)  # Convert 'content' to string

In [ ]:
import sys
print("Python version:", sys.version)

In [ ]:
reviews_fg.insert(df)

Uploading Dataframe: 100.00% |██████████| Rows 29994/29994 | Elapsed Time: 00:05 | Remaining Time: 00:00


Launching job: reviews_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1185941/jobs/named/reviews_1_offline_fg_materialization/executions


(Job('reviews_1_offline_fg_materialization', 'SPARK'), None)

In [ ]:
print(df.info())  # Check column names and data types
print(df.head())  # Preview the first few rows


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29994 entries, 0 to 29993
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   reviewId           29994 non-null  object        
 1   content            29994 non-null  object        
 2   score              29994 non-null  int64         
 3   at                 29994 non-null  datetime64[ns]
 4   thumbsUpCount      29994 non-null  int64         
 5   augmented_comment  29994 non-null  object        
 6   sentiment          29994 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 1.6+ MB
None
                               reviewId  \
0  b35813f3-bf30-43ae-8364-a66e8e6263fa   
1  92b8d4da-0de0-457f-bd4a-886916b53ef5   
2  fd78f35e-a309-4dd4-91cd-e878d3259291   
3  d8d12d28-ef26-48f6-ae76-d4e3b906f88f   
4  89679fff-c646-4c9b-b3e6-847a403bbcdb   

                                             content  score  \
0

In [ ]:
df.to_excel('reviews_labelled.xlsx', index=False, engine='openpyxl')